In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

# --- 1. Load Pretrained EfficientNet ---
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model initially

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# --- 2. Build Custom Classification Head ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')  # 10 classes
])

In [ ]:
# --- 3. Compile the Model ---
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


In [ ]:
# --- 4. Image Data Generators ---
train_datagen = ImageDataGenerator(
    rescale=1.0/255, rotation_range=20, horizontal_flip=True, zoom_range=0.2
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/AI/Datasets/train/train", target_size=(224, 224), batch_size=32, class_mode="sparse"
)
val_generator = val_datagen.flow_from_directory(
    "/content/drive/MyDrive/AI/Datasets/test/test", target_size=(224, 224), batch_size=32, class_mode="sparse"
)

Found 13914 images belonging to 10 classes.
Found 3483 images belonging to 10 classes.


In [ ]:
# --- 5. Train the Model ---
history = model.fit(train_generator, validation_data=val_generator, epochs=10)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4947s 11s/step - accuracy: 0.1713 - loss: 2.2417 - val_accuracy: 0.1789 - val_loss: 2.2131
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1580s 4s/step - accuracy: 0.1800 - loss: 2.2211 - val_accuracy: 0.1809 - val_loss: 2.2095
Epoch 3/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1568s 4s/step - accuracy: 0.1720 - loss: 2.2233 - val_accuracy: 0.1809 - val_loss: 2.2232
Epoch 4/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1608s 4s/step - accuracy: 0.1792 - loss: 2.2146 - val_accuracy: 0.1809 - val_loss: 2.2107
Epoch 5/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1637s 4s/step - accuracy: 0.1847 - loss: 2.2120 - val_accuracy: 0.1809 - val_loss: 2.2090
Epoch 6/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1600s 4s/step - accuracy: 0.1796 - loss: 2.2167 - val_accuracy: 0.1809 - val_loss: 2.2096
Epoch 7/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1599s 4s/step - accuracy: 0.1759 - loss: 2.2174 - val_accuracy: 0.1809 - val_loss: 2.2093
Epoch 8/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1672s 4s/step - accuracy: 0.1792 - loss: 2.2109 -

In [ ]:
model.save("/content/drive/MyDrive/AI/models/efficientnet_model_no_tune.h5")
import json

# Save history
with open("/content/drive/MyDrive/AI/models/training_history.json", "w") as f:
    json.dump(history.history, f)


In [ ]:
# --- 6. Fine-Tune (Unfreeze Base Model) ---
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

history_finetune = model.fit(train_generator, validation_data=val_generator, epochs=10)


Epoch 1/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 5065s 12s/step - accuracy: 0.2222 - loss: 2.4693 - val_accuracy: 0.2667 - val_loss: 2.1272
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 5002s 11s/step - accuracy: 0.4762 - loss: 1.6163 - val_accuracy: 0.2862 - val_loss: 2.0602
Epoch 3/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4996s 11s/step - accuracy: 0.6100 - loss: 1.2322 - val_accuracy: 0.3207 - val_loss: 1.9463
Epoch 4/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4977s 11s/step - accuracy: 0.7005 - loss: 0.9536 - val_accuracy: 0.2038 - val_loss: 2.2699
Epoch 5/10
255/435 ━━━━━━━━━━━━━━━━━━━━ 32:36 11s/step - accuracy: 0.7523 - loss: 0.7831

In [ ]:
# --- 7. Save the Model ---
model.save("/content/drive/MyDrive/AI/models/efficientnet_model_3.h5")
import json

# Save history
with open("/content/drive/MyDrive/AI/models/history_finetune.json", "w") as f:
    json.dump(history_finetune.history, f)


In [ ]:

# --- 8. Plot Training Accuracy & Loss ---
def plot_training_history(history, history_finetune):
    acc = history.history['accuracy'] + history_finetune.history['accuracy']
    val_acc = history.history['val_accuracy'] + history_finetune.history['val_accuracy']
    loss = history.history['loss'] + history_finetune.history['loss']
    val_loss = history.history['val_loss'] + history_finetune.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 5))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend()
    plt.title("Training & Validation Accuracy")

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend()
    plt.title("Training & Validation Loss")

    plt.show()

plot_training_history(history, history_finetune)

In [ ]:

# --- 9. Model Evaluation on Validation Data ---
val_generator.reset()
y_true = val_generator.classes
y_pred_probs = model.predict(val_generator)
y_pred = np.argmax(y_pred_probs, axis=1)


In [ ]:
# --- 10. Classification Report ---
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))


In [ ]:
# --- 11. Confusion Matrix ---
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

plot_confusion_matrix(y_true, y_pred, list(val_generator.class_indices.keys()))


# Load model + fine tune

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

In [ ]:
model = load_model("/content/drive/MyDrive/AI/models/efficientnet_model_no_tune.h5")

In [ ]:
base_model = model.layers[0]
base_model.trainable = True

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=20, horizontal_flip=True, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/AI/Datasets/train/train", target_size=(224, 224), batch_size=32, class_mode="sparse"
)
val_generator = val_datagen.flow_from_directory(
    "/content/drive/MyDrive/AI/Datasets/test/test", target_size=(224, 224), batch_size=32, class_mode="sparse"
)


Found 13914 images belonging to 10 classes.
Found 3483 images belonging to 10 classes.


In [ ]:
history_finetune = model.fit(train_generator, validation_data=val_generator, epochs=10)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 5070s 11s/step - accuracy: 0.2276 - loss: 2.4721 - val_accuracy: 0.1266 - val_loss: 19.0154
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4522s 10s/step - accuracy: 0.5237 - loss: 1.5188 - val_accuracy: 0.6492 - val_loss: 1.2805
Epoch 3/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4536s 10s/step - accuracy: 0.6642 - loss: 1.0808 - val_accuracy: 0.7775 - val_loss: 0.7566
Epoch 4/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4552s 10s/step - accuracy: 0.7429 - loss: 0.8378 - val_accuracy: 0.8229 - val_loss: 0.5767
Epoch 5/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4530s 10s/step - accuracy: 0.7903 - loss: 0.6681 - val_accuracy: 0.8498 - val_loss: 0.4713
Epoch 6/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4500s 10s/step - accuracy: 0.8126 - loss: 0.5739 - val_accuracy: 0.8708 - val_loss: 0.4093
Epoch 7/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4539s 10s/step - accuracy: 0.8417 - loss: 0.4952 - val_accuracy: 0.8840 - val_loss: 0.3597
Epoch 8/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 4591s 11s/step - accuracy: 0.8521 - loss: 

In [ ]:
model.save("/content/drive/MyDrive/AI/models/efficientnet_model_3.h5")


In [ ]:
with open("/content/drive/MyDrive/AI/models/history_finetune.json", "w") as f:
    json.dump(history_finetune.history, f)
